In [1]:
#semisupervised learning (check 12/9: #1)


In [1]:
#imports
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import sys
#sys.path.append("../BerkeleySETI/Messy-Notebooks")
sys.path.append("../FinalClassifier/data")
import os

import data
from data import loaders
from data import phasefold as pf
import lightkurve as lk
import warnings
import glob
import pickle

%load_ext autoreload
%autoreload 2

In [2]:
#set up
warnings.filterwarnings("ignore", category=RuntimeWarning) #fset = pd.read_csv('sect6data.csv')
data_dir = "/mnt/disks/lcs/"
data.mount_drive(data_dir)
data_dir = "/mnt/disks/lcs/tess-goddard-lcs/"

Disk mounted


In [3]:
#import sector 6 features
!gcsfuse --implicit-dirs tess-goddard-lcs /home/jupyter/mountpoint
datafile = "/home/jupyter/mountpoint/dataproducts/db.v2.h5"
scores6 = pd.read_hdf(datafile, f'S6/scores')
features6 = pd.read_hdf(datafile, f'S6/features')

2022/12/12 23:44:03.147650 Start gcsfuse/0.39.2 (Go version go1.17.5) for app "" using mount point: /home/jupyter/mountpoint
2022/12/12 23:44:03.381965 Opening GCS connection...
2022/12/12 23:44:03.639806 Mounting file system "tess-goddard-lcs"...
2022/12/12 23:44:03.733951 File system has been successfully mounted.


In [4]:
#list of ebs that we will use for our training data
txt_file = open("data/ebs_s6.txt", "r")
file_content = txt_file.read()
content_list = file_content.split("\n")
txt_file.close()
eb = [eval(i) for i in content_list]

#list of eb tics
eb

[235231072,
 301185875,
 301185877,
 234169663,
 89016404,
 80836949,
 140336965,
 156501953,
 282503011,
 715766691,
 301185891,
 436619704,
 39354700,
 220140809,
 156241056,
 285000038,
 285000042,
 234065348,
 265899106,
 39838127,
 234878308,
 43144900,
 206794153,
 265715239,
 207004915,
 235548197,
 247681431,
 36288537,
 119461266,
 234169642,
 237590010,
 39667795,
 235552142,
 79635172,
 369673753,
 280212098,
 266193357,
 143215473,
 234926584,
 445955531,
 712487688,
 279900791,
 387690877,
 42991740,
 438545375,
 92009366,
 159086692,
 247556517,
 284199394,
 231161386,
 445955532,
 445955536,
 206840164,
 206840192,
 234158088,
 308387164,
 60242904,
 714514062,
 139476794,
 139476806,
 308169172,
 159057212,
 309908733,
 349480507,
 42885762,
 206840156,
 206850953,
 234146623,
 308169197,
 60242900,
 220149256,
 231959604,
 42991745,
 140209204,
 237647557,
 264594192,
 264594193,
 264594194,
 281157204,
 138904948,
 280304732,
 437918915,
 97568437,
 247681430,
 369681

In [5]:
f6eb = features6.loc[eb]

In [6]:
#list of ebs from villanova (should be exhaustive in tmcl), which we will use to find our noneb dataset
tmclebs = pd.read_csv('data/allsectorstmclebs.csv')

ref = pd.read_csv(data_dir+f"sector6lookup.csv")
tmcl = ["2_min_cadence" in fn for fn in ref.Filename]
reftmcl = ref[tmcl]
z = ~reftmcl.TIC_ID.isin(tmclebs.TIC_ID.values)
f6nonebs = reftmcl[z].sample(frac=1)

In [7]:
#features for our noneb
f6noneb = features6.loc[f6nonebs.TIC_ID.values]
f6noneb

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
271974161,0.032344,0.998953,1.046139e-03,0.032378,1.0,0.0,1.0,1.0,0.000214,0.017685,...,0.429319,0.166933,0.261269,0.399501,0.542412,0.765537,1.000000,0.676874,0.164,0.096686
153096232,0.032653,0.998959,1.066221e-03,0.032687,1.0,0.0,1.0,1.0,0.000211,0.009523,...,0.481481,0.117481,0.210124,0.323335,0.459885,0.703940,0.989737,2.730455,0.159,0.175096
96864159,0.032855,0.998827,1.079457e-03,0.032894,1.0,0.0,1.0,1.0,0.001378,0.102698,...,0.391819,0.139847,0.255965,0.403570,0.562599,0.760437,1.000000,0.569661,0.166,0.084078
238952126,0.032760,0.998918,1.073206e-03,0.032795,1.0,0.0,1.0,1.0,0.000328,0.026664,...,0.398496,0.149777,0.270440,0.387015,0.549708,0.762141,1.000000,0.690990,0.163,0.097369
364590678,0.033008,0.998905,1.089526e-03,0.033044,1.0,0.0,1.0,1.0,0.000308,0.023726,...,0.353326,0.148870,0.273767,0.407492,0.529275,0.771921,1.000000,0.637985,0.167,0.144009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119941661,0.137997,1.009735,1.904313e-02,0.136666,9.0,5.0,1.0,6.0,0.002767,0.433883,...,0.897275,0.131290,0.276604,0.404973,0.579085,0.768881,2.408121,35.245277,-0.346,0.000000
157132601,0.033465,0.998002,1.119904e-03,0.033532,1.0,0.0,1.0,1.0,0.003827,0.058810,...,0.434876,0.099732,0.198445,0.343029,0.529981,0.764657,1.000000,0.446853,0.248,0.405265
231811265,0.033079,0.998902,1.094249e-03,0.033116,1.0,0.0,1.0,1.0,0.000248,0.019001,...,0.434830,0.135211,0.244902,0.376715,0.522371,0.770689,1.000000,0.699603,0.165,0.083301
242065922,0.000311,0.999994,9.658428e-08,0.000311,232.0,0.0,4.0,4.0,0.000191,0.012060,...,0.354839,0.163036,0.310488,0.438526,0.591955,0.775606,0.002951,0.683374,0.399,-0.034867


In [67]:
#run this line to read in feats.csv
feats = pd.read_csv('data/feats.csv', index_col = 0)

In [9]:
train, test = train_test_split(np.arange(len(feats)), train_size=0.8)
labels = []

for i in range(len(f6noneb)):
    labels.append('NonEB')
for i in range(len(f6noneb),len(f6eb)+len(f6noneb)):
    labels.append('EB')
    
X_trainfull = feats.iloc[train]
X_testfull = feats.iloc[test]

y_train = []
y_test = []
for i in train:
    y_train.append(labels[i])
for i in test:
    y_test.append(labels[i])
    
y_train = pd.Series(data = y_train, index = train)
y_test = pd.Series(data = y_test, index = test)

In [10]:
newy_train = []
for i in (y_train.index.values):
    if (y_train[i]=='NonEB'):
        newy_train.append(0)
    else:
        newy_train.append(1)
y_train = pd.Series(data = newy_train, index = train)

newy_test = []
for i in (y_test.index.values):
    if (y_test[i]=='NonEB'):
        newy_test.append(0)
    else:
        newy_test.append(1)
y_test = pd.Series(data = newy_test, index = test)

In [11]:
X_train = X_trainfull
X_test = X_testfull

In [12]:
RF = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators = 500, max_depth = 15,random_state=31415))
RF.fit(X_train,y_train)
y_predRF = RF.predict(X_test)
print(accuracy_score(y_test,y_predRF)) # Fraction of classifications that were correct
print(precision_score(y_test,y_predRF,pos_label=1))

0.9995160900072586
0.9980430528375733


In [13]:
#sector you want to classify
sector = 6

In [14]:
#change the variable toClassify to data that you want to classify
toClassify = pd.read_hdf(datafile, f'S{sector}/features')
toClassifyWProb = pd.read_hdf(datafile, f'S{sector}/features')
refsect = pd.read_csv(data_dir+f"sector{sector}lookup.csv")

In [15]:
y_pred = RF.predict(toClassify)
#rslt_df.to_csv(f'Sector{sector}/predSector{sector}.csv')

In [16]:
y_predprob = RF.predict_proba(toClassify)

In [17]:
y_predprob

array([[1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       ...,
       [1.   , 0.   ],
       [0.998, 0.002],
       [1.   , 0.   ]])

In [18]:
#y_probzero = prediction confidence probability of zero
#y_probone = prediction confidence probability of one
y_probzero = []
y_probone = []
for i in range(len(y_predprob)):
    y_probzero.append(y_predprob[i][0])
    y_probone.append(y_predprob[i][1])

In [19]:
toClassifyWProb['pred'] = y_pred
toClassifyWProb['probzero'] = y_probzero
toClassifyWProb['probone'] = y_probone
rslt_df = toClassifyWProb[toClassifyWProb['pred'] == 1]

In [20]:
toClassifyWProb

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat,pred,probzero,probone
251269189,0.000269,0.999981,7.222776e-08,0.000269,242.0,0.0,5.0,5.0,0.000153,0.008087,...,0.378293,0.536652,0.765845,0.002046,0.534761,0.487,0.101711,0,1.000,0.000
11199201,0.000340,1.000012,1.157584e-07,0.000340,301.0,1.0,0.0,1.0,0.000230,0.013218,...,0.407784,0.566966,0.763643,0.001540,0.996224,-0.094,0.289609,0,1.000,0.000
11197342,0.000355,1.000025,1.257538e-07,0.000355,287.0,0.0,0.0,0.0,0.000218,0.009656,...,0.384835,0.534202,0.795165,0.001338,0.776569,-0.238,0.395487,0,1.000,0.000
234271852,0.002851,0.999978,8.128245e-06,0.002851,260.0,0.0,1.0,1.0,0.001765,0.142535,...,0.415781,0.578310,0.784964,0.033248,0.714904,0.104,0.229952,0,1.000,0.000
237736917,0.001683,1.000044,2.831918e-06,0.001683,291.0,1.0,0.0,1.0,0.001107,0.077765,...,0.396938,0.561477,0.785257,0.009305,1.231992,-0.166,0.044972,0,1.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306910194,0.002299,0.999605,5.285637e-06,0.002300,183.0,1.0,7.0,8.0,0.001228,0.028972,...,0.426980,0.553811,0.802176,0.017737,0.812900,0.612,0.226680,0,1.000,0.000
306808426,0.001467,0.999996,2.152518e-06,0.001467,248.0,4.0,0.0,4.0,0.000953,0.016892,...,0.447909,0.600444,0.766037,0.012969,2.198081,-0.096,0.305667,0,1.000,0.000
306911868,0.001238,0.999867,1.532303e-06,0.001238,327.0,0.0,0.0,0.0,0.000926,0.012549,...,0.467407,0.628315,0.809456,0.003785,0.477304,0.199,0.241496,0,1.000,0.000
306912997,0.001271,1.000168,1.615076e-06,0.001271,213.0,2.0,0.0,2.0,0.000576,0.014940,...,0.291748,0.418907,0.627517,0.008711,1.379476,-0.288,0.296004,0,0.998,0.002


In [27]:
r1eb = toClassifyWProb[toClassifyWProb['probzero'] == 0]
r1noneb = toClassifyWProb[toClassifyWProb['probzero'] == 1]

In [28]:
r1eb

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat,pred,probzero,probone
309908721,0.029504,0.988698,0.000871,0.029842,118.0,0.0,0.0,0.0,0.001948,0.206244,...,0.044064,0.140639,0.617406,0.114579,0.396873,1.458,0.691977,1,0.0,1.0
63192395,0.071090,0.973933,0.005054,0.072993,105.0,0.0,1.0,1.0,0.002644,0.344462,...,0.025820,0.151856,0.661622,1.000000,0.027656,1.444,0.825547,1,0.0,1.0
144695651,0.063551,0.982520,0.004039,0.064682,66.0,0.0,25.0,25.0,0.001549,0.280385,...,0.017892,0.028141,0.058099,0.375209,0.054150,1.335,0.857458,1,0.0,1.0
153166843,0.087738,0.969763,0.007698,0.090474,99.0,0.0,1.0,1.0,0.007618,0.799157,...,0.061483,0.084089,0.528471,0.976360,0.057769,1.459,0.817655,1,0.0,1.0
33715938,0.081920,0.973406,0.006711,0.084158,87.0,0.0,1.0,1.0,0.001975,0.460906,...,0.017202,0.030043,0.381459,1.000000,0.019142,1.483,0.866047,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229385088,0.141951,0.943489,0.020150,0.150453,100.0,0.0,0.0,0.0,0.017111,0.763913,...,0.110653,0.229720,0.523877,0.559366,0.061073,1.484,0.902321,1,0.0,1.0
160327122,0.094427,0.962853,0.008917,0.098071,111.0,0.0,1.0,1.0,0.007664,1.030180,...,0.074687,0.305458,0.607667,0.996557,0.045621,1.393,0.781077,1,0.0,1.0
247786809,0.061939,0.977156,0.003836,0.063387,92.0,0.0,0.0,0.0,0.014046,0.463133,...,0.158483,0.295839,0.458853,0.264218,0.162767,1.329,0.776228,1,0.0,1.0
234715460,0.063095,0.975122,0.003981,0.064705,123.0,0.0,0.0,0.0,0.003705,0.330387,...,0.043510,0.206853,0.698132,0.270624,0.053536,1.486,0.735242,1,0.0,1.0


In [29]:
r1noneb

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat,pred,probzero,probone
251269189,0.000269,0.999981,7.222776e-08,0.000269,242.0,0.0,5.0,5.0,0.000153,0.008087,...,0.378293,0.536652,0.765845,0.002046,0.534761,0.487,0.101711,0,1.0,0.0
11199201,0.000340,1.000012,1.157584e-07,0.000340,301.0,1.0,0.0,1.0,0.000230,0.013218,...,0.407784,0.566966,0.763643,0.001540,0.996224,-0.094,0.289609,0,1.0,0.0
11197342,0.000355,1.000025,1.257538e-07,0.000355,287.0,0.0,0.0,0.0,0.000218,0.009656,...,0.384835,0.534202,0.795165,0.001338,0.776569,-0.238,0.395487,0,1.0,0.0
234271852,0.002851,0.999978,8.128245e-06,0.002851,260.0,0.0,1.0,1.0,0.001765,0.142535,...,0.415781,0.578310,0.784964,0.033248,0.714904,0.104,0.229952,0,1.0,0.0
237736917,0.001683,1.000044,2.831918e-06,0.001683,291.0,1.0,0.0,1.0,0.001107,0.077765,...,0.396938,0.561477,0.785257,0.009305,1.231992,-0.166,0.044972,0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151473503,0.002332,1.000226,5.440286e-06,0.002332,195.0,8.0,0.0,8.0,0.001115,0.020033,...,0.293623,0.408402,0.653968,0.015756,1.234452,-0.581,0.302418,0,1.0,0.0
306910194,0.002299,0.999605,5.285637e-06,0.002300,183.0,1.0,7.0,8.0,0.001228,0.028972,...,0.426980,0.553811,0.802176,0.017737,0.812900,0.612,0.226680,0,1.0,0.0
306808426,0.001467,0.999996,2.152518e-06,0.001467,248.0,4.0,0.0,4.0,0.000953,0.016892,...,0.447909,0.600444,0.766037,0.012969,2.198081,-0.096,0.305667,0,1.0,0.0
306911868,0.001238,0.999867,1.532303e-06,0.001238,327.0,0.0,0.0,0.0,0.000926,0.012549,...,0.467407,0.628315,0.809456,0.003785,0.477304,0.199,0.241496,0,1.0,0.0


In [49]:
r1remaining = toClassifyWProb[~(toClassifyWProb.index.isin(r1noneb.index.values))]

In [50]:
r1remaining

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat,pred,probzero,probone
177174910,0.035846,0.998096,0.001285,0.035914,3.0,0.0,3.0,3.0,0.001178,0.064407,...,0.279126,0.385077,0.569916,1.000000,0.667108,0.292,-0.009942,0,0.996,0.004
25133286,0.965037,1.029756,0.931297,0.937152,2.0,1.0,0.0,1.0,0.003954,4.161035,...,0.071482,0.195225,0.367643,29.175622,95.010291,-0.161,0.127373,0,0.862,0.138
237591126,0.007902,0.999940,0.000062,0.007903,266.0,0.0,1.0,1.0,0.005083,0.393820,...,0.392708,0.552803,0.771836,0.071210,0.588164,0.106,0.073576,0,0.998,0.002
151290873,0.006460,0.998300,0.000042,0.006471,342.0,0.0,0.0,0.0,0.004237,0.041125,...,0.496726,0.695855,0.882386,0.018352,0.725418,0.513,0.510237,0,0.998,0.002
41533230,0.051553,0.997018,0.002658,0.051707,236.0,0.0,1.0,1.0,0.038502,0.335776,...,0.658501,0.805917,0.912231,0.995325,0.465517,0.196,0.852387,0,0.992,0.008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25077726,0.033038,0.998926,0.001092,0.033073,1.0,0.0,1.0,1.0,0.000619,0.021017,...,0.250069,0.424777,0.707092,1.000000,0.667015,0.165,0.323875,0,0.998,0.002
438060613,0.009085,1.001493,0.000083,0.009071,326.0,0.0,0.0,0.0,0.005923,0.058709,...,0.395920,0.634077,0.828394,0.024429,0.668926,-0.412,0.711753,0,0.998,0.002
437886874,0.009973,0.999927,0.000099,0.009973,258.0,0.0,0.0,0.0,0.006061,0.081949,...,0.377408,0.522265,0.743122,0.036319,0.538968,0.147,0.429114,0,0.998,0.002
247867421,0.002231,1.000491,0.000005,0.002230,69.0,16.0,0.0,16.0,0.000555,0.024631,...,0.231346,0.309161,0.458192,0.020814,1.601120,-0.945,0.105977,0,0.984,0.016


In [51]:
#these don't have the probabilities, so we add these to feats compared to r1eb and r1noneb
r1ebToAdd = toClassify[toClassifyWProb.index.isin(r1eb.index.values)]
r1nonebToAdd = toClassify[toClassifyWProb.index.isin(r1noneb.index.values)]

In [68]:
feats = feats.append(r1ebToAdd)
#feats = feats.append(r1nonebToAdd)

In [69]:
#retrain for round 1
train, test = train_test_split(np.arange(len(feats)), train_size=0.8)
labels = []

for i in range(len(f6noneb)):
    labels.append('NonEB')
for i in range(len(f6noneb),len(f6eb)+len(f6noneb)):
    labels.append('EB')
for i in range(len(f6eb)+len(f6noneb), len(f6eb)+len(f6noneb)+len(r1ebToAdd)):
    labels.append('EB')
#for i in range(len(f6eb)+len(f6noneb)+len(r1ebToAdd), len(f6eb)+len(f6noneb)+len(r1ebToAdd)+len(r1nonebToAdd)):
    #labels.append('NonEB')
    
X_trainfull = feats.iloc[train]
X_testfull = feats.iloc[test]

y_train = []
y_test = []
for i in train:
    y_train.append(labels[i])
for i in test:
    y_test.append(labels[i])
    
y_train = pd.Series(data = y_train, index = train)
y_test = pd.Series(data = y_test, index = test)

In [70]:
newy_train = []
for i in (y_train.index.values):
    if (y_train[i]=='NonEB'):
        newy_train.append(0)
    else:
        newy_train.append(1)
y_train = pd.Series(data = newy_train, index = train)

newy_test = []
for i in (y_test.index.values):
    if (y_test[i]=='NonEB'):
        newy_test.append(0)
    else:
        newy_test.append(1)
y_test = pd.Series(data = newy_test, index = test)

In [71]:
X_train = X_trainfull
X_test = X_testfull

In [72]:
RF1 = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators = 500, max_depth = 15,random_state=31415))
RF1.fit(X_train,y_train)
y_predRF = RF1.predict(X_test)
print(accuracy_score(y_test,y_predRF)) # Fraction of classifications that were correct
print(precision_score(y_test,y_predRF,pos_label=1))

0.9997827031725337
0.9989637305699481


In [73]:
#sector you want to classify
sector = 6

In [97]:
#change the variable toClassify to data that you want to classify
toClassify = pd.read_hdf(datafile, f'S{sector}/features')
refsect = pd.read_csv(data_dir+f"sector{sector}lookup.csv")

In [98]:
y_pred = RF1.predict(toClassify)
#toClassify['pred'] = y_pred
#rslt_df = toClassify[toClassify['pred'] == 1]
#rslt_df

KeyError: 'pred'

In [99]:
y_predprob = RF1.predict_proba(toClassify)
y_predprob

array([[1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       ...,
       [1.   , 0.   ],
       [0.998, 0.002],
       [1.   , 0.   ]])

In [100]:
#y_probzero = prediction confidence probability of zero
#y_probone = prediction confidence probability of one
y_probzero = []
y_probone = []
for i in range(len(y_predprob)):
    y_probzero.append(y_predprob[i][0])
    y_probone.append(y_predprob[i][1])

In [101]:
toClassify['pred'] = y_pred
toClassify['probzero'] = y_probzero
toClassify['probone'] = y_probone
rslt_df = toClassify[toClassify['pred'] == 1]

In [102]:
rslt_df

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat,pred,probzero,probone
141126559,0.129283,0.966157,0.016714,0.133812,225.0,0.0,1.0,1.0,0.078338,1.612755,...,0.382946,0.535229,0.865685,0.966863,0.332072,0.809,0.866650,1,0.405100,0.594900
31303242,0.071903,0.982412,0.005170,0.073190,58.0,0.0,22.0,22.0,0.000670,0.214146,...,0.008720,0.011854,0.017514,0.998374,0.007386,1.206,0.673295,1,0.052000,0.948000
260161144,0.066176,0.978258,0.004379,0.067647,97.0,0.0,1.0,1.0,0.000965,0.448632,...,0.010310,0.022724,0.484134,0.998295,0.010259,1.435,0.850411,1,0.000000,1.000000
156241056,0.070821,0.972317,0.005016,0.072837,112.0,0.0,0.0,0.0,0.002759,0.628637,...,0.025548,0.119360,0.613303,0.305047,0.067491,1.550,0.875323,1,0.000000,1.000000
309908721,0.029504,0.988698,0.000871,0.029842,118.0,0.0,0.0,0.0,0.001948,0.206244,...,0.044064,0.140639,0.617406,0.114579,0.396873,1.458,0.691977,1,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250470344,0.026891,0.991369,0.000723,0.027125,89.0,0.0,19.0,19.0,0.001047,0.109420,...,0.026788,0.049050,0.398950,0.154673,0.023941,1.480,0.707522,1,0.000000,1.000000
220420534,0.046102,0.987261,0.002125,0.046696,78.0,0.0,1.0,1.0,0.003164,0.226328,...,0.069424,0.185201,0.418781,0.999090,0.079798,1.097,0.696240,1,0.437867,0.562133
33716637,0.071296,0.971362,0.005083,0.073398,105.0,0.0,1.0,1.0,0.015595,0.396347,...,0.247192,0.397989,0.628234,0.997895,0.113634,1.258,0.850790,1,0.004000,0.996000
251258191,0.025978,0.992660,0.000675,0.026171,71.0,0.0,26.0,26.0,0.000588,0.072821,...,0.015233,0.022041,0.049144,0.135237,0.020763,1.469,0.755226,1,0.002000,0.998000


In [91]:
rslt_df = rslt_df[rslt_df['probone'] == 1]

In [96]:
rslt_df

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat,pred,probzero,probone
291334195,0.066176,0.978258,0.004379,0.067647,97.0,0.0,1.0,1.0,0.000965,0.448632,...,0.010310,0.022724,0.484134,0.998295,0.010259,1.435,0.850411,1,0.0,1.0
436423802,0.070821,0.972317,0.005016,0.072837,112.0,0.0,0.0,0.0,0.002759,0.628637,...,0.025548,0.119360,0.613303,0.305047,0.067491,1.550,0.875323,1,0.0,1.0
251232498,0.029504,0.988698,0.000871,0.029842,118.0,0.0,0.0,0.0,0.001948,0.206244,...,0.044064,0.140639,0.617406,0.114579,0.396873,1.458,0.691977,1,0.0,1.0
140343098,0.071090,0.973933,0.005054,0.072993,105.0,0.0,1.0,1.0,0.002644,0.344462,...,0.025820,0.151856,0.661622,1.000000,0.027656,1.444,0.825547,1,0.0,1.0
229195593,0.063551,0.982520,0.004039,0.064682,66.0,0.0,25.0,25.0,0.001549,0.280385,...,0.017892,0.028141,0.058099,0.375209,0.054150,1.335,0.857458,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63202928,0.070926,0.978580,0.005031,0.072479,78.0,0.0,6.0,6.0,0.000697,0.354885,...,0.007349,0.012435,0.323605,0.948263,0.089656,1.423,0.728172,1,0.0,1.0
159090432,0.094427,0.962853,0.008917,0.098071,111.0,0.0,1.0,1.0,0.007664,1.030180,...,0.074687,0.305458,0.607667,0.996557,0.045621,1.393,0.781077,1,0.0,1.0
251139907,0.061939,0.977156,0.003836,0.063387,92.0,0.0,0.0,0.0,0.014046,0.463133,...,0.158483,0.295839,0.458853,0.264218,0.162767,1.329,0.776228,1,0.0,1.0
256879784,0.063095,0.975122,0.003981,0.064705,123.0,0.0,0.0,0.0,0.003705,0.330387,...,0.043510,0.206853,0.698132,0.270624,0.053536,1.486,0.735242,1,0.0,1.0


In [92]:
import random
tic = rslt_df.index.values
random.shuffle(tic)

In [93]:
ref_subset = ref[ref['TIC_ID'].isin(tic[:250])]

In [94]:
s6_sub_files = data_dir+ref_subset.Filename.values
lcc = loaders.LightCurveCollection(ref_subset.Sector.values,
                                   ref_subset.Camera.values,
                                   s6_sub_files,
                            )
lcc.useCpus = 4
lcc.load_all_lcs()

In [95]:
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(f"TestNewDataset/250ofrslt_dfR1.pdf") as pdf:
    plt.cla()
    for i in range(250):
        lcc.lcs[i].scatter()
        pdf.savefig()
        plt.close();
plt.close()